In [44]:
import json
import transformers
import torch

In [2]:
# import json file returned by document retrivial 
with open("data.json") as f:
    data = json.load(f)

In [3]:
data = data["body_text"]

Perform zero-shot learning on the data based on pre-trained Bert (pretrained on SQuAD version 1)

In [4]:
from transformers import BertForQuestionAnswering

In [5]:
# Bert QA pretrained Model
model = BertForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

In [6]:
# Bert Tokenizer
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

In [39]:
question = "What do we know about COVID-19 risk factors?"
answer_text = data[:536] 
# this is just for demo, in final report, we will split our thesis into paragraphs

In [40]:
answer_text

'Severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2), identified in December 2019, is the cause of the illness named COVID-19. 1, 2 With more than 249,000 confirmed cases and more than 28,700 deaths by 20 th August 2020, Spain remains one of the European countries most severely affected by the ongoing COVID-19 pandemic. 3, 4 Spain also established a universal screening programme for pregnancies in light of the higher disease exposure. We observed that obstetric intervention may influence the clinical course of the disease.'

Tokenize our data

In [41]:
input_ids = tokenizer.encode(question, answer_text)
print("the input has a total of {:} tokens.".format(len(input_ids)))

the input has a total of 132 tokens.


In [42]:
tokens = tokenizer.convert_ids_to_tokens(input_ids)

for token, id in zip(tokens, input_ids):
    
    if id == tokenizer.sep_token_id:
        print("")
    
    print("{:<12} {:>6}".format(token, id))
    
    if id == tokenizer.sep_token_id:
        print("")

[CLS]           101
what           2054
do             2079
we             2057
know           2113
about          2055
co             2522
##vid         17258
-              1011
19             2539
risk           3891
factors        5876
?              1029

[SEP]           102

severe         5729
acute         11325
respiratory   16464
syndrome       8715
corona        21887
##virus       23350
2              1016
(              1006
sar           18906
##s            2015
-              1011
co             2522
##v            2615
-              1011
2              1016
)              1007
,              1010
identified     4453
in             1999
december       2285
2019          10476
,              1010
is             2003
the            1996
cause          3426
of             1997
the            1996
illness        7355
named          2315
co             2522
##vid         17258
-              1011
19             2539
.              1012
1              1015
,              101

In [43]:
sep_index = input_ids.index(tokenizer.sep_token_id)

num_seg_a = sep_index + 1
num_seg_b = len(input_ids) - num_seg_a
# this is for the bertQA required input (to mask the question part)
segment_ids = [0]*num_seg_a + [1]*num_seg_b

assert len(segment_ids) == len(input_ids)

Run through model

In [45]:
# start scores and end_scores means the probability of every tokens being start of answer and end of answer
start_scores, end_scores = model(torch.tensor([input_ids]),
                                token_type_ids = torch.tensor([segment_ids]))

In [46]:
answer_start = torch.argmax(start_scores)
answer_end = torch.argmax(end_scores)

answer = " ".join(tokens[answer_start:answer_end+1])

print("Answer:   " + answer)

Answer:   we observed that ob ##ste ##tric intervention may influence the clinical course of the disease .
